# German Credit Risk Analysis - Power BI Data Preparation
This notebook prepares clean data for Power BI dashboard creation

## Setup and Data Loading

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
pd.set_option("display.max_columns", None)

In [ ]:
# Load raw data
df_raw = pd.read_csv("../data/raw/german_credit_data.csv")
print(f"Original data shape: {df_raw.shape}")
df_raw.head()

## Step 1: Initial Data Inspection

In [5]:
print("Missing values per column:")
print(df_raw.isna().sum())
print("\nData types:")
print(df_raw.dtypes)

Missing values per column:
Unnamed: 0            0
Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

Data types:
Unnamed: 0           int64
Age                  int64
Sex                 object
Job                  int64
Housing             object
Saving accounts     object
Checking account    object
Credit amount        int64
Duration             int64
Purpose             object
Risk                object
dtype: object


## Step 2: Clean and Prepare Data for Power BI

In [6]:
# Create a copy for Power BI preparation
df_powerbi = df_raw.copy()

# Remove unnecessary column
if 'Unnamed: 0' in df_powerbi.columns:
    df_powerbi = df_powerbi.drop(columns=['Unnamed: 0'])

print(f"Cleaned data shape: {df_powerbi.shape}")

Cleaned data shape: (1000, 10)


## Step 3: Handle Missing Values

In [7]:
# Fill missing values with 'Unknown' for categorical columns
df_powerbi['Saving accounts'] = df_powerbi['Saving accounts'].fillna('Unknown')
df_powerbi['Checking account'] = df_powerbi['Checking account'].fillna('Unknown')

print("Missing values after cleaning:")
print(df_powerbi.isna().sum())

Missing values after cleaning:
Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
Risk                0
dtype: int64


## Step 4: Create Business-Friendly Labels

In [8]:
# Map Job codes to descriptive names
job_mapping = {
    0: 'Unskilled',
    1: 'Semi-Skilled',
    2: 'Skilled',
    3: 'Highly Skilled'
}
df_powerbi['Job_Category'] = df_powerbi['Job'].map(job_mapping)

# Create numeric risk score (1=Good, 0=Bad)
risk_mapping = {'good': 1, 'bad': 0}
df_powerbi['Risk_Score'] = df_powerbi['Risk'].map(risk_mapping)

print("Job categories created:")
print(df_powerbi['Job_Category'].value_counts())

Job categories created:
Job_Category
Skilled           630
Semi-Skilled      200
Highly Skilled    148
Unskilled          22
Name: count, dtype: int64


## Step 5: Create Categorical Bins for Analysis

In [9]:
# Age Groups
df_powerbi['Age_Group'] = pd.cut(
    df_powerbi['Age'],
    bins=[0, 25, 35, 50, 100],
    labels=['Young (18-25)', 'Adult (26-35)', 'Middle-Aged (36-50)', 'Senior (51+)']
)

# Credit Amount Tiers
df_powerbi['Credit_Tier'] = pd.cut(
    df_powerbi['Credit amount'],
    bins=[0, 2000, 5000, 10000, 20000],
    labels=['Small (<2K)', 'Medium (2K-5K)', 'Large (5K-10K)', 'Very Large (10K+)']
)

# Duration Categories
df_powerbi['Duration_Category'] = pd.cut(
    df_powerbi['Duration'],
    bins=[0, 12, 24, 36, 100],
    labels=['Short (<1yr)', 'Medium (1-2yr)', 'Long (2-3yr)', 'Very Long (3yr+)']
)

print("\nAge Group distribution:")
print(df_powerbi['Age_Group'].value_counts())
print("\nCredit Tier distribution:")
print(df_powerbi['Credit_Tier'].value_counts())


Age Group distribution:
Age_Group
Adult (26-35)          398
Middle-Aged (36-50)    299
Young (18-25)          190
Senior (51+)           113
Name: count, dtype: int64

Credit Tier distribution:
Credit_Tier
Small (<2K)          432
Medium (2K-5K)       380
Large (5K-10K)       148
Very Large (10K+)     40
Name: count, dtype: int64


## Step 6: Create Risk Indicator Flags

In [10]:
# Calculate median for comparisons
median_credit = df_powerbi['Credit amount'].median()

# Create binary flags
df_powerbi['High_Amount_Flag'] = (df_powerbi['Credit amount'] > median_credit).astype(int)
df_powerbi['Long_Duration_Flag'] = (df_powerbi['Duration'] > 24).astype(int)
df_powerbi['Young_Applicant_Flag'] = (df_powerbi['Age'] < 30).astype(int)

print(f"High Amount Flag (>{median_credit}): {df_powerbi['High_Amount_Flag'].sum()} records")
print(f"Long Duration Flag (>24 months): {df_powerbi['Long_Duration_Flag'].sum()} records")
print(f"Young Applicant Flag (<30 years): {df_powerbi['Young_Applicant_Flag'].sum()} records")

High Amount Flag (>2319.5): 500 records
Long Duration Flag (>24 months): 230 records
Young Applicant Flag (<30 years): 371 records


## Step 7: Create Business Metrics

In [11]:
# Estimated monthly payment
df_powerbi['Estimated_Monthly_Payment'] = (df_powerbi['Credit amount'] / df_powerbi['Duration']).round(2)

# Credit to Age ratio
df_powerbi['Credit_Per_Age'] = (df_powerbi['Credit amount'] / df_powerbi['Age']).round(2)

print("\nEstimated Monthly Payment statistics:")
print(df_powerbi['Estimated_Monthly_Payment'].describe())


Estimated Monthly Payment statistics:
count    1000.000000
mean      167.686950
std       153.490973
min        24.060000
25%        89.602500
50%       130.335000
75%       206.185000
max      2482.670000
Name: Estimated_Monthly_Payment, dtype: float64


## Step 8: Create Exposure Level Classification

In [12]:
def classify_exposure(row):
    """Classify credit exposure based on amount and duration"""
    if row['Credit amount'] > 5000 and row['Duration'] > 24:
        return 'High Exposure'
    elif row['Credit amount'] < 2000 and row['Duration'] < 12:
        return 'Low Exposure'
    else:
        return 'Medium Exposure'

df_powerbi['Exposure_Level'] = df_powerbi.apply(classify_exposure, axis=1)

print("\nExposure Level distribution:")
print(df_powerbi['Exposure_Level'].value_counts())


Exposure Level distribution:
Exposure_Level
Medium Exposure    746
Low Exposure       130
High Exposure      124
Name: count, dtype: int64


## Step 9: Add Metadata Columns

In [13]:
# Add record ID
df_powerbi['Record_ID'] = range(1, len(df_powerbi) + 1)

# Add data load timestamp
df_powerbi['Data_Load_Date'] = datetime.now().strftime('%Y-%m-%d')

print(f"\nTotal records: {len(df_powerbi)}")
print(f"Data load date: {df_powerbi['Data_Load_Date'].iloc[0]}")


Total records: 1000
Data load date: 2026-01-06


## Step 10: Reorder Columns for Power BI

In [14]:
# Define column order for better organization
column_order = [
    'Record_ID',
    'Data_Load_Date',
    # Demographics
    'Age',
    'Age_Group',
    'Sex',
    'Job',
    'Job_Category',
    'Housing',
    # Financial Status
    'Saving accounts',
    'Checking account',
    # Loan Details
    'Credit amount',
    'Credit_Tier',
    'Duration',
    'Duration_Category',
    'Purpose',
    # Calculated Metrics
    'Estimated_Monthly_Payment',
    'Credit_Per_Age',
    # Risk Assessment
    'Risk',
    'Risk_Score',
    'Exposure_Level',
    # Flags
    'High_Amount_Flag',
    'Long_Duration_Flag',
    'Young_Applicant_Flag'
]

df_powerbi = df_powerbi[column_order]

print("Final column structure:")
print(df_powerbi.columns.tolist())

Final column structure:
['Record_ID', 'Data_Load_Date', 'Age', 'Age_Group', 'Sex', 'Job', 'Job_Category', 'Housing', 'Saving accounts', 'Checking account', 'Credit amount', 'Credit_Tier', 'Duration', 'Duration_Category', 'Purpose', 'Estimated_Monthly_Payment', 'Credit_Per_Age', 'Risk', 'Risk_Score', 'Exposure_Level', 'High_Amount_Flag', 'Long_Duration_Flag', 'Young_Applicant_Flag']


## Step 11: Data Quality Check

In [15]:
print("=" * 60)
print("FINAL DATA QUALITY REPORT")
print("=" * 60)
print(f"\nTotal Records: {len(df_powerbi)}")
print(f"Total Columns: {len(df_powerbi.columns)}")
print(f"\nMissing Values:")
print(df_powerbi.isna().sum().sum(), "(Should be 0)")
print(f"\nDuplicate Rows: {df_powerbi.duplicated().sum()} (Should be 0)")
print(f"\nData Types:")
print(df_powerbi.dtypes.value_counts())
print("\n" + "=" * 60)

FINAL DATA QUALITY REPORT

Total Records: 1000
Total Columns: 23

Missing Values:
0 (Should be 0)

Duplicate Rows: 0 (Should be 0)

Data Types:
int64       9
object      9
float64     2
category    1
category    1
category    1
Name: count, dtype: int64



## Step 12: Preview Final Dataset

In [16]:
print("First 5 rows of Power BI ready dataset:")
df_powerbi.head()

First 5 rows of Power BI ready dataset:


,Record_ID,Data_Load_Date,Age,Age_Group,Sex,Job,Job_Category,Housing,Saving accounts,Checking account,Credit amount,Credit_Tier,Duration,Duration_Category,Purpose,Estimated_Monthly_Payment,Credit_Per_Age,Risk,Risk_Score,Exposure_Level,High_Amount_Flag,Long_Duration_Flag,Young_Applicant_Flag
0,1,2026-01-06,67,Senior (51+),male,2,Skilled,own,Unknown,little,1169,Small (<2K),6,Short (<1yr),radio/TV,194.83,17.45,good,1,Low Exposure,0,0,0
1,2,2026-01-06,22,Young (18-25),female,2,Skilled,own,little,moderate,5951,Large (5K-10K),48,Very Long (3yr+),radio/TV,123.98,270.50,bad,0,High Exposure,1,1,1
2,3,2026-01-06,49,Middle-Aged (36-50),male,1,Semi-Skilled,own,little,Unknown,2096,Medium (2K-5K),12,Short (<1yr),education,174.67,42.78,good,1,Medium Exposure,0,0,0
3,4,2026-01-06,45,Middle-Aged (36-50),male,2,Skilled,free,little,little,7882,Large (5K-10K),42,Very Long (3yr+),furniture/equipment,187.67,175.16,good,1,High Exposure,1,1,0
4,5,2026-01-06,53,Senior (51+),male,2,Skilled,free,little,little,4870,Medium (2K-5K),24,Medium (1-2yr),car,202.92,91.89,bad,0,Medium Exposure,1,0,0


In [17]:
print("\nSample statistics:")
df_powerbi.describe()


Sample statistics:


,Record_ID,Age,Job,Credit amount,Duration,Estimated_Monthly_Payment,Credit_Per_Age,Risk_Score,High_Amount_Flag,Long_Duration_Flag,Young_Applicant_Flag
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000
mean,500.500000,35.546000,1.904000,3271.258000,20.903000,167.686950,99.495940,0.700000,0.50000,0.230000,0.371000
std,288.819436,11.375469,0.653614,2822.736876,12.058814,153.490973,91.251618,0.458487,0.50025,0.421043,0.483314
min,1.000000,19.000000,0.000000,250.000000,4.000000,24.060000,6.100000,0.000000,0.00000,0.000000,0.000000
25%,250.750000,27.000000,2.000000,1365.500000,12.000000,89.602500,40.100000,0.000000,0.00000,0.000000,0.000000
50%,500.500000,33.000000,2.000000,2319.500000,18.000000,130.335000,68.630000,1.000000,0.50000,0.000000,0.000000
75%,750.250000,42.000000,2.000000,3972.250000,24.000000,206.185000,127.610000,1.000000,1.00000,0.000000,1.000000
max,1000.000000,75.000000,3.000000,18424.000000,72.000000,2482.670000,745.380000,1.000000,1.00000,1.000000,1.000000


## Step 13: Export to CSV for Power BI

In [ ]:
# Export to CSV
output_filename = '../data/Processed/german_credit_data_powerbi.csv'
df_powerbi.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"✓ Data exported successfully to: {output_filename}")
print(f"✓ File ready for Power BI import")
print(f"\nFile contains:")
print(f"  - {len(df_powerbi)} records")
print(f"  - {len(df_powerbi.columns)} columns")
print(f"  - 0 missing values")
print(f"  - Clean, business-friendly labels")

## Step 14: Create Data Dictionary

In [ ]:
# Create data dictionary
data_dictionary = pd.DataFrame({
    'Column_Name': df_powerbi.columns,
    'Data_Type': df_powerbi.dtypes.astype(str),
    'Description': [
        'Unique record identifier',
        'Date when data was processed',
        'Applicant age in years',
        'Age category (Young/Adult/Middle-Aged/Senior)',
        'Gender (male/female)',
        'Job skill level code (0-3)',
        'Job skill level description',
        'Housing status (own/rent/free)',
        'Savings account level (little/moderate/rich/quite rich/Unknown)',
        'Checking account level (little/moderate/rich/Unknown)',
        'Loan amount requested',
        'Credit amount category',
        'Loan duration in months',
        'Loan duration category',
        'Purpose of the loan',
        'Estimated monthly payment amount',
        'Credit amount divided by age',
        'Credit risk classification (good/bad)',
        'Numeric risk score (1=Good, 0=Bad)',
        'Risk exposure classification (Low/Medium/High)',
        'Flag for above-median credit amount (1=Yes, 0=No)',
        'Flag for duration >24 months (1=Yes, 0=No)',
        'Flag for age <30 years (1=Yes, 0=No)'
    ],
    'Sample_Values': [
        str(df_powerbi[col].head(3).tolist()) for col in df_powerbi.columns
    ]
})

# Save data dictionary
data_dictionary.to_csv('../data/Processed/data_dictionary_powerbi.csv', index=False)
print("\n✓ Data dictionary created: ../data/Processed/data_dictionary_powerbi.csv")
print("\nData Dictionary Preview:")
data_dictionary.head(10)

## Summary Statistics for Dashboard Planning

In [20]:
print("\n" + "="*60)
print("KEY METRICS FOR POWER BI DASHBOARD")
print("="*60)

print(f"\n1. RISK DISTRIBUTION:")
print(df_powerbi['Risk'].value_counts())
print(f"   Bad Risk Rate: {(df_powerbi['Risk']=='bad').sum()/len(df_powerbi)*100:.1f}%")

print(f"\n2. EXPOSURE LEVELS:")
print(df_powerbi['Exposure_Level'].value_counts())

print(f"\n3. AVERAGE METRICS:")
print(f"   Avg Credit Amount: ${df_powerbi['Credit amount'].mean():.2f}")
print(f"   Avg Duration: {df_powerbi['Duration'].mean():.1f} months")
print(f"   Avg Age: {df_powerbi['Age'].mean():.1f} years")
print(f"   Avg Monthly Payment: ${df_powerbi['Estimated_Monthly_Payment'].mean():.2f}")

print(f"\n4. TOP LOAN PURPOSES:")
print(df_powerbi['Purpose'].value_counts().head(5))

print(f"\n5. JOB CATEGORY DISTRIBUTION:")
print(df_powerbi['Job_Category'].value_counts())

print("\n" + "="*60)


KEY METRICS FOR POWER BI DASHBOARD

1. RISK DISTRIBUTION:
Risk
good    700
bad     300
Name: count, dtype: int64
   Bad Risk Rate: 30.0%

2. EXPOSURE LEVELS:
Exposure_Level
Medium Exposure    746
Low Exposure       130
High Exposure      124
Name: count, dtype: int64

3. AVERAGE METRICS:
   Avg Credit Amount: $3271.26
   Avg Duration: 20.9 months
   Avg Age: 35.5 years
   Avg Monthly Payment: $167.69

4. TOP LOAN PURPOSES:
Purpose
car                    337
radio/TV               280
furniture/equipment    181
business                97
education               59
Name: count, dtype: int64

5. JOB CATEGORY DISTRIBUTION:
Job_Category
Skilled           630
Semi-Skilled      200
Highly Skilled    148
Unskilled          22
Name: count, dtype: int64



SyntaxError: invalid syntax (3203778117.py, line 1)

## Suggested Power BI Dashboard Components

### KPIs to Display:
1. **Total Loans**: Count of Record_ID
2. **Total Credit Amount**: Sum of Credit amount
3. **Average Credit Amount**: Average of Credit amount
4. **Bad Risk Rate**: Count of Risk='bad' / Total count
5. **Average Monthly Payment**: Average of Estimated_Monthly_Payment

### Recommended Visualizations:
1. **Risk Distribution Pie Chart**: Risk by count
2. **Credit Amount by Purpose**: Bar chart
3. **Age Group vs Risk**: Stacked bar chart
4. **Exposure Level Distribution**: Donut chart
5. **Credit Tier by Job Category**: Clustered bar chart
6. **Duration Category Distribution**: Column chart
7. **Risk Trends**: Line chart (if time data available)
8. **Credit Amount Distribution**: Histogram

### Filters/Slicers:
- Sex
- Housing
- Job_Category
- Age_Group
- Credit_Tier
- Duration_Category
- Purpose
- Risk
- Exposure_Level